In [2]:
%pip install PyEMD
%pip install EMD-signal

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import matplotlib as plt
from matplotlib import pyplot
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import os
import pywt
from scipy import signal
import math
from math import *
from PyEMD import EMD
# from PyEMD.CEEMDAN import CEEMDAN
from numpy import *
import sys
from biosppy.signals import ecg
import statistics

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import wden
import baseline

from sklearn.metrics import classification_report

from keras.models import load_model

ModuleNotFoundError: No module named 'PyEMD'

In [ ]:
# Load model
#model = load_model('Models/best_model.08-0.17.h5')
# model = load_model('LSTM_Models/best_model.09-0.19.h5')
model = load_model('Models/best_model.01-0.67.h5')
print('Model loaded')

Model loaded


In [9]:
# Load target dataset
data = pd.read_csv("PTB/record001_001_Myocardial Infarction.csv", header = 0)

In [ ]:
# Process data to be similar to input training data
signal_array = []
file_array = []
for column in data:
    if column != 'vx' and column != 'vy' and column != 'vz':
        df = data[column]
        arrays = np.array(df)
        arrays = baseline.baseline_corr(arrays)
        peak = signal.argrelmax(arrays, order = 450)
        start = peak[0][2] - 200
        end = peak[0][2] + 400
        arrays = arrays[start:end]
        arrays = wden.wdenoise(arrays, 'sqtwolog', 'soft', 'mln', 4, 'sym7')
        arrays = signal.resample(arrays, 150)
        file_array.append(arrays)

signal_array.append(file_array)
signal_array = np.asarray(signal_array).reshape(-1, 150, 12)

print ('Input Loaded')

In [14]:
# Process data to be similar to input training data
signal_array = []
file_array = []

#data = baseline.baseline_corr(data)
#data = wden.wdenoise(data, 'sqtwolog', 'soft', 'mln', 4, 'sym7')
df = np.array(data['v4'])
    
peaks = ecg.christov_segmenter(df, 250)[0]
for peak in range (1, len(peaks) - 1):
    segment_array = []
    for column in data:
        df = np.array(data[column])
        segment = df[peaks[peak]-50:peaks[peak]+100]
        segment_array.append(segment)  
    signal_array.append(segment_array)

signal_array = np.asarray(signal_array).reshape(-1, 150, 12)

print ('Input Loaded')
print (signal_array.shape[0])

Input Loaded
39


In [5]:
# Transform data into 12 sections
input1 = signal_array[:, :, 0].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input2 = signal_array[:, :, 1].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input3 = signal_array[:, :, 2].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input4 = signal_array[:, :, 3].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input5 = signal_array[:, :, 4].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input6 = signal_array[:, :, 5].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input7 = signal_array[:, :, 6].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input8 = signal_array[:, :, 7].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input9 = signal_array[:, :, 8].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input10 = signal_array[:, :, 9].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input11 = signal_array[:, :, 10].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input12 = signal_array[:, :, 11].reshape(signal_array.shape[0], signal_array.shape[1], 1)

In [6]:
# Predictor
y_pred_test = model.predict([input1, input2, input3, input4, input5, input6, input7, input8, input9, input10, input11, input12], verbose = 1)
# Take the class with the highest probability from the test predictions
prediction = np.argmax(y_pred_test, axis=1)

labels = ['Healthy', 'Myocardial Infarction']

print (labels[int(prediction)])

1/1 [==============================] - 1s 694ms/step
Healthy


In [28]:
predictions = []
for i in range (signal_array.shape[0]):
    input1 = signal_array[i, :, 0].reshape(1, signal_array.shape[1], 1)
    input2 = signal_array[i, :, 1].reshape(1, signal_array.shape[1], 1)
    input3 = signal_array[i, :, 2].reshape(1, signal_array.shape[1], 1)
    input4 = signal_array[i, :, 3].reshape(1, signal_array.shape[1], 1)
    input5 = signal_array[i, :, 4].reshape(1, signal_array.shape[1], 1)
    input6 = signal_array[i, :, 5].reshape(1, signal_array.shape[1], 1)
    input7 = signal_array[i, :, 6].reshape(1, signal_array.shape[1], 1)
    input8 = signal_array[i, :, 7].reshape(1, signal_array.shape[1], 1)
    input9 = signal_array[i, :, 8].reshape(1, signal_array.shape[1], 1)
    input10 = signal_array[i, :, 9].reshape(1, signal_array.shape[1], 1)
    input11 = signal_array[i, :, 10].reshape(1, signal_array.shape[1], 1)
    input12 = signal_array[i, :, 11].reshape(1, signal_array.shape[1], 1)
    
    # Predictor
    y_pred_test = model.predict([input1, input2, input3, input4, input5, input6, input7, input8, input9, input10, input11, input12], verbose = 1)
    # Take the class with the highest probability from the test predictions
    prediction = np.argmax(y_pred_test, axis=1)
    
    labels = ['Healthy', 'Myocardial Infarction']
    predictions.append(labels[int(prediction)])
    #print (labels[int(prediction)])
    

final_prediction = statistics.mode(predictions) 
print (final_prediction)

1/1 [==============================] - 0s 101ms/step
Myocardial Infarction


In [11]:
print (signal_array.shape)

(1, 150, 12)
